In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to the UTKFace directories
utkface_dir1 = 'E:/p2/age/archive (5)/UTKFace'
utkface_dir2 = 'E:/p2/age/archive (5)/utkface_aligned_cropped/UTKFace'

# Define age ranges and labels
age_labels = {
    'Child': (0, 12),
    'Teen': (13, 19),
    'Young Adult': (20, 35),
    'Adult': (36, 60),
    'Senior': (61, 100)
}

# Function to extract metadata from filename
def extract_metadata_from_filename(filename):
    try:
        age, gender, race, _ = filename.split('_')
        return int(age), int(gender), int(race)
    except ValueError:
        return None, None, None

# Function to get age label based on age
def get_age_label(age):
    for label, (min_age, max_age) in age_labels.items():
        if min_age <= age <= max_age:
            return label
    return None

# Function to process the directory and extract metadata
def process_utkface_directory(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            age, gender, race = extract_metadata_from_filename(filename)
            if age is not None:
                age_label = get_age_label(age)
                if age_label:  # Ensure label is valid
                    full_path = os.path.join(directory, filename)
                    data.append({'age': age, 'age_label': age_label, 'gender': gender, 'race': race, 'full_path': full_path})
    return pd.DataFrame(data)

# Load the datasets from both directories
print("Loading datasets...")
utkface_df1 = process_utkface_directory(utkface_dir1)
utkface_df2 = process_utkface_directory(utkface_dir2)

# Combine the datasets
utkface_df = pd.concat([utkface_df1, utkface_df2])
print(f"Combined dataset size: {utkface_df.shape[0]}")

# Sample a subset of the data to handle memory issues
subset_size = 10000  # Adjust this number based on your memory capacity
subset_df = utkface_df.sample(subset_size, random_state=42)

# Define the data generators
print("Setting up data generators...")
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1.0/255)

# Setup the training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=subset_df,
    x_col='full_path',
    y_col='age_label',
    target_size=(128, 128),
    batch_size=32,
    subset='training',
    class_mode='categorical',
    classes=list(age_labels.keys())
)

# Setup the validation generator
val_generator = val_datagen.flow_from_dataframe(
    dataframe=subset_df,
    x_col='full_path',
    y_col='age_label',
    target_size=(128, 128),
    batch_size=32,
    subset='validation',
    class_mode='categorical',
    classes=list(age_labels.keys())
)

# Check validation samples
print(f"Validation samples found: {val_generator.samples}")

# Build the CNN model
print("Building the model...")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(age_labels), activation='softmax')  # Output layer for categorical classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model using generators
print("Training the model...")
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  # Increased epochs for better training
)

# Save the model
print("Saving the model...")
model.save('age_estimation_model.h5')
print("Model saved as 'age_estimation_model.h5'")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')

# Check if 'val_accuracy' exists before plotting
if 'val_accuracy' in history.history:
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training History')
plt.show()


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the model
print("Loading the model...")
model = load_model('age_estimation_model.h5')
print("Model loaded.")

# Define age ranges and labels
age_labels = {
    'Child': (0, 12),
    'Teen': (13, 19),
    'Young Adult': (20, 35),
    'Adult': (36, 60),
    'Senior': (61, 100)
}

def get_age_label(age):
    for label, (min_age, max_age) in age_labels.items():
        if min_age <= age <= max_age:
            return label
    return 'Unknown'

def preprocess_image(image, target_size=(128, 128)):
    image = cv2.resize(image, target_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image to [0, 1]
    image = np.expand_dims(image, axis=0)
    return image

def get_age_range_label(prediction):
    labels = list(age_labels.keys())
    return labels[np.argmax(prediction)]

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
print("Starting webcam...")
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Extract the face region
        face = frame[y:y+h, x:x+w]
        
        # Preprocess the face for prediction
        processed_face = preprocess_image(face)
        
        # Predict the age range
        prediction = model.predict(processed_face)[0]
        age_range_label = get_age_range_label(prediction)
        
        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"Age Range: {age_range_label}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Age Estimation', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()
print("Webcam closed.")
